# Mission

Le projet porte sur StackOverFlow, la célèbre plateforme de questions et réponses populaire pour les développeurs.  
La mission est de développer un système qui génère les suggestions de tags lorsqu'un utilisateur pose une question sur le site. 
Le projet se décline sous forme de 4 notebooks:  
- un notebook avec un test du wrapper StackAPI pour récupérer 50 questions StackOverFlow     
- un notebook d'exploration et de pré-traitement des questions récupérées   
- un notebook avec une approche non supervisée de génération de tags     
- un notebook avec une approche supervisée de génération de tags    

# Génération de tags - Modèle non supervisé

1. [Importation des bibliothèques nécessaires](#Importation-des-bibliothèques-nécessaires)  
    Des bibliothèques comme pandas, numpy, matplotlib, seaborn, scipy, et gensim sont importées pour le traitement des données, la visualisation, et l'analyse statistique.

2. [Chargement et Prétraitement des Données](#Chargement-et-Prétraitement-des-Données)  
    Le dataset `questions_cleaned_stackoverflow.csv` est chargé dans un DataFrame pandas. Les colonnes `Tags_list` et `Question_list` sont créées en séparant les chaînes de caractères des colonnes `Tags` et `Question_bow` respectivement, en utilisant la virgule comme séparateur.

3. [Analyse de la Fréquence des Tags](#Analyse-de-la-Fréquence-des-Tags)  
    Les fréquences des tags sont calculées et les 200 tags les plus fréquents sont identifiés. Le DataFrame est ensuite filtré pour ne conserver que les questions dont tous les tags sont parmi ces 200 tags les plus fréquents.

4. [Vectorisation des Données Textuelles](#Vectorisation-des-Données-Textuelles)  
    Les données textuelles sont vectorisées en utilisant `TfidfVectorizer` et `MultiLabelBinarizer` pour préparer les données pour la modélisation.

5. [Division des Données en Ensembles d'Entraînement et de Test](#Division-des-Données-en-Ensembles-d'Entraînement-et-de-Test)  
    Les données sont divisées en ensembles d'entraînement et de test, avec une répartition de 70% pour l'entraînement et 30% pour le test.

6. [Modélisation avec LDA (Latent Dirichlet Allocation)](#Modélisation-avec-LDA-(Latent-Dirichlet-Allocation))  
    Un modèle LDA est construit sur les données pour découvrir des topics dans le texte. Le modèle est évalué en termes de perplexité et de cohérence. La visualisation des topics est également effectuée avec `pyLDAvis`.

7. [Association Document-Topic-Tag](#Association-Document-Topic-Tag)  
    Les relations entre les documents, les topics, et les tags sont explorées, et une tentative de prédiction des tags en fonction des topics est effectuée.

8. [Évaluation des Prédictions](#Évaluation-des-Prédictions)  
    Les prédictions sont évaluées en termes d'Accuracy, F1 Score, Jaccard Similarity Score, Recall et Precision.

### Importation des bibliothèques nécessaires

Installation des dépendances suivantes :  
ipykernel==6.26.0  
pandas==2.1.3    
gensim==4.3.2  
scikit-learn==1.3.2  
nltk==3.8.1  
pyLDAvis==3.4.1  




In [1]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import scipy.stats as st

### Chargement et Prétraitement des Données

In [3]:
questions = pd.read_csv("datasets/questions_cleaned_stackoverflow.csv")

questions['Tags_list'] = questions['Tags'].str.split(',')
questions['Question_list'] = questions['Question_bow'].str.split(',')
questions.shape


(30582, 17)

### Analyse de la Fréquence des Tags


In [4]:
import pandas as pd
from collections import Counter
from itertools import chain

# Calculer la fréquence des tags
tag_frequencies = Counter(chain.from_iterable(questions['Tags_list']))

# Identifier les 50 tags les plus fréquents
top_200_tags = {tag for tag, count in tag_frequencies.most_common(200)}

# Filtrer le DataFrame
filtered_df = questions[questions['Tags_list'].apply(lambda tags: all(tag in top_200_tags for tag in tags))]
questions = filtered_df
print(questions.shape)
top_200_tags


(1074, 17)


{'.net',
 '.net-core',
 'ajax',
 'algorithm',
 'amazon-web-services',
 'android',
 'android-fragments',
 'android-layout',
 'android-studio',
 'angular',
 'angularjs',
 'apache',
 'arrays',
 'asp.net',
 'asp.net-core',
 'asp.net-mvc',
 'asp.net-web-api',
 'assembly',
 'async-await',
 'asynchronous',
 'authentication',
 'azure',
 'bash',
 'boost',
 'browser',
 'c',
 'c#',
 'c++',
 'c++11',
 'c++14',
 'c++17',
 'caching',
 'casting',
 'clang',
 'class',
 'cocoa-touch',
 'compiler-construction',
 'compiler-optimization',
 'concurrency',
 'cordova',
 'css',
 'data-structures',
 'database',
 'date',
 'datetime',
 'debugging',
 'delphi',
 'dependency-injection',
 'dictionary',
 'django',
 'docker',
 'dom',
 'eclipse',
 'ecmascript-6',
 'encryption',
 'entity-framework',
 'excel',
 'exception',
 'express',
 'file',
 'firebase',
 'firefox',
 'floating-point',
 'flutter',
 'forms',
 'function',
 'functional-programming',
 'g++',
 'garbage-collection',
 'gcc',
 'generics',
 'git',
 'google-chrom

### Vectorisation des Données Textuelles


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

X = questions["Question_list"]
y = questions["Tags_list"]

def list_to_string(lst):
    return ' '.join(lst)

vectorizer = TfidfVectorizer(analyzer="word",
                             max_df=.6,
                             min_df=0.005,
                             tokenizer=None,
                             preprocessor=list_to_string,
                             stop_words=None,
                             lowercase=False)

vectorizer.fit(X)
X_tfidf = vectorizer.transform(X)
print("Format de X (questions): {}".format(X_tfidf.shape))

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(y)
y_binarized = multilabel_binarizer.transform(y)
print("Format de y (tags): {}".format(y_binarized.shape))


Format de X (questions): (1074, 847)
Format de y (tags): (1074, 196)


### Division des Données en Ensembles d'Entraînement et de Test


In [6]:
from sklearn.model_selection import train_test_split

# Create train and test split (30%)
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_binarized,
                                                    test_size=0.3, random_state=8)
print("X_train : {}".format(X_train.shape))
print("X_test : {}".format(X_test.shape))
print("y_train : {}".format(y_train.shape))
print("y_test : {}".format(y_test.shape))


X_train : (751, 847)
X_test : (323, 847)
y_train : (751, 196)
y_test : (323, 196)


### Modélisation avec LDA (Latent Dirichlet Allocation)


Le principe sera le suivant:
Le modèle LDA est utilisé pour découvrir des sujets (topics) dans le texte des questions. Une fois les sujets découverts, chaque document (dans notre cas, une question) est associé à un ou plusieurs sujets en fonction des mots qu'il contient.  
Le modèle LDA en lui-même est un modèle non supervisé. Il est utilisé pour découvrir des sujets cachés sans avoir besoin de labels pour l'entraînement.  

La partie de prédiction des tags en fonction des sujets et de l'association document-sujet rendra notre système global semi-supervisé. On n'entraîne pas LDA avec des labels, mais on utilise ensuite des labels (tags) pour explorer et évaluer les relations entre sujets et tags.

In [ ]:
import gensim
from gensim import corpora
import pyLDAvis.gensim_models
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
id2word = corpora.Dictionary(X)
id2word.filter_extremes(no_below=4, no_above=0.6, keep_n=None)

texts = X  
corpus = [id2word.doc2bow(text) for text in texts]  
print(corpus[:1])
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]
# Build LDA model
full_lda_model = gensim.models.ldamulticore\
                    .LdaMulticore(corpus=corpus,
                                  id2word=id2word,
                                  num_topics=10,
                                  random_state=8,
                                  per_word_topics=True,
                                  workers=4)
print('\nPerplexity: ', full_lda_model.log_perplexity(corpus))


La perplexité est souvent utilisée pour évaluer la qualité des modèles probabilistes de langage. Ici on a une qualité faible, puisque négative.

In [9]:
from gensim.models import CoherenceModel

coherence_model_lda = CoherenceModel(model=full_lda_model, 
                                     texts=texts, 
                                     dictionary=id2word, 
                                     coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)



Coherence Score:  0.35305569104617085


La cohérence mesure la similitude sémantique entre les mots les plus probables pour chaque thème. Les valeurs de cohérence varient entre -1 et 1, où une valeur plus élevée indique une meilleure cohérence entre les mots dans les thèmes identifiés par le modèle LDA. Ici la cohérence est modérée.

In [10]:
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
%matplotlib inline

gensimvis.prepare(full_lda_model, corpus, id2word)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.018986 -0.003429       1        1  14.320078
5      0.002759  0.021827       2        1  13.996318
1      0.018586  0.017674       3        1  11.137909
3      0.033075 -0.008310       4        1  10.751262
2     -0.012666 -0.039711       5        1   9.857986
6     -0.013332  0.021832       6        1   9.594596
4      0.004013  0.032563       7        1   9.367523
0     -0.011070  0.037606       8        1   8.632499
7     -0.114238 -0.031574       9        1   6.264988
9      0.073888 -0.048479      10        1   6.076841, topic_info=             Term        Freq       Total Category  logprob  loglift
118        memory  394.000000  394.000000  Default  30.0000  30.0000
303         image  312.000000  312.000000  Default  29.0000  29.0000
194   application  324.000000  324.000000  Default  28.0000  28.0000
21       function  492.000000  492.000000  Default  27.0000  27.0000
85        request  176.000000  176.000000  Default  26.0000  26.0000
..            ...         ...         ...      ...      ...      ...
11           data   19.733903  356.075393  Topic10  -4.8256  -0.0921
168         array   16.113347  277.853677  Topic10  -5.0283  -0.0468
59       question   15.846070  309.995914  Topic10  -5.0450  -0.1730
37           time   16.786051  463.664209  Topic10  -4.9874  -0.5179
5            case   13.137764  244.332929  Topic10  -5.2324  -0.1224

[720 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
974       1  0.130488   ....
974       2  0.130488   ....
974       4  0.130488   ....
974       6  0.130488   ....
974       8  0.260976   ....
...     ...       ...    ...
845       3  0.423925   type
845       7  0.211962   type
439       2  0.479803   unit
439       3  0.159934   unit
439       9  0.319869   unit

[3059 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 6, 2, 4, 3, 7, 5, 1, 8, 10])

### Association Document-Topic-Tag


L'association se fait en 2 temps:

    - On convertit la sortie du modèle LDA en un DataFrame doc_topic, où chaque ligne représente un document et chaque colonne un sujet, avec des valeurs indiquant la probabilité d'association entre les documents et les sujets.
    - On calcule ensuite une matrice topic_tag en effectuant une multiplication matricielle entre la transposition de doc_topic et y_binarized, afin d'obtenir une relation entre les sujets et les tags basée sur la présence des tags dans les documents associés à chaque sujet.

In [11]:
doc_topic = pd.DataFrame(full_lda_model\
                             .get_document_topics(corpus,
                                                  minimum_probability=0))
for topic in doc_topic.columns:
    doc_topic[topic] = doc_topic[topic].apply(lambda x : x[1])

print('document/tag : ', y_binarized.shape)
print('document/topic : ', doc_topic.shape)
# Matricial multiplication with Document / Topics transpose
topic_tag = np.matmul(doc_topic.T, y_binarized)
topic_tag.shape


document/tag :  (1074, 196)
document/topic :  (1074, 10)


(10, 196)

### Évaluation des Prédictions


Les tags ont été numérotés par MultiLabelBinarizer, en voici la liste en utilisant l'attribut classes_  de MultiLabelBinarizer.

In [12]:
tags_with_number = list(enumerate(multilabel_binarizer.classes_))
tags_df = pd.DataFrame(tags_with_number, columns=['Number', 'Tag'])
print(tags_df)


     Number                  Tag
0         0                 .net
1         1            .net-core
2         2                 ajax
3         3            algorithm
4         4  amazon-web-services
..      ...                  ...
191     191                  wpf
192     192                  x86
193     193               x86-64
194     194                xcode
195     195                  xml

[196 rows x 2 columns]


In [13]:
y_results = pd.DataFrame(y)
y_results["best_topic"] = doc_topic.idxmax(axis=1).values

df_y_bin = pd.DataFrame(y_binarized)
df_dict = dict(
    list(
        df_y_bin.groupby(df_y_bin.index)
    )
)

tags_num = []
for k, v in df_dict.items():
    check = v.columns[(v == 1).any()]
    tags_num.append(check.to_list())

y_results["y_true"] = tags_num
y_results.sample(5)


,Tags_list,best_topic,y_true
25751,"[c#, .net, c++, c, vb.net]",6,"[0, 25, 26, 27, 176]"
810,"[c++, performance, assembly, optimization, x86]",1,"[17, 27, 121, 127, 192]"
3992,"[c, gcc, assembly, x86, compiler-optimization]",5,"[17, 25, 37, 69, 192]"
28280,"[jquery, html, css, google-chrome, firefox]",7,"[40, 61, 71, 76, 91]"
20285,"[java, javascript, c++, c, algorithm]",8,"[3, 25, 27, 87, 89]"


In [14]:
y_results.sample(10)

,Tags_list,best_topic,y_true
18400,"[c++, visual-studio, visual-studio-2010, lambd...",3,"[27, 28, 95, 180, 181]"
14009,"[c#, asp.net, .net, wcf, exception]",6,"[0, 13, 26, 57, 185]"
26757,"[c++, python, algorithm, optimization, scipy]",5,"[3, 27, 121, 131, 144]"
29568,"[c++, arrays, pointers, language-lawyer, c++17]",3,"[12, 27, 30, 97, 129]"
15054,"[javascript, android, jquery, html, angularjs]",0,"[5, 10, 76, 89, 91]"
25258,"[c++, templates, recursion, lambda, c++14]",1,"[27, 29, 95, 138, 163]"
25760,"[c#, .net, generics, asp.net-core, .net-core]",5,"[0, 1, 14, 26, 70]"
23216,"[c#, java, c++, c, algorithm]",3,"[3, 25, 26, 27, 87]"
12228,"[javascript, angularjs, reactjs, react-native,...",3,"[10, 51, 89, 136, 137]"
28317,"[java, c#, python, algorithm, language-agnostic]",8,"[3, 26, 87, 96, 131]"


In [15]:
list_tag = []
for row in y_results.itertuples():
    best_topic = row.best_topic
    row_tags = list(topic_tag.iloc[best_topic]\
                    .sort_values(ascending=False)[0:5].index)
    list_tag.append(row_tags)
    
y_results["y_pred"] = list_tag
y_results.sample(3)


,Tags_list,best_topic,y_true,y_pred
351,"[javascript, jquery, html, css, twitter-bootst...",4,"[40, 76, 89, 91, 166]","[27, 87, 26, 69, 89]"
16692,"[visual-c++, optimization, assembly, x86, sse]",8,"[17, 121, 157, 179, 192]","[89, 27, 25, 127, 91]"
13414,"[c#, .net, string, memory-management, performa...",1,"[0, 26, 109, 127, 160]","[26, 127, 27, 0, 87]"


In [16]:
# Création d'un dictionnaire de mappage des numéros aux noms des tags
tag_mapping = {i: tag for i, tag in enumerate(multilabel_binarizer.classes_)}

def numbers_to_names(tag_numbers_list):
    return [tag_mapping[tag_number] for tag_number in tag_numbers_list]

# Application de la conversion aux colonnes 'y_true' et 'y_pred' de y_results
y_results['y_tags_true'] = y_results['y_true'].apply(numbers_to_names)
y_results['y_tags_pred'] = y_results['y_pred'].apply(numbers_to_names)
y_results.sample(5)


,Tags_list,best_topic,y_true,y_pred,y_tags_true,y_tags_pred
27803,"[javascript, php, jquery, sql-server, ajax]",0,"[2, 89, 91, 128, 155]","[89, 91, 76, 27, 128]","[ajax, javascript, jquery, php, sql-server]","[javascript, jquery, html, c++, php]"
663,"[c#, javascript, html, angularjs, asp.net-web-...",4,"[10, 16, 26, 76, 89]","[27, 87, 26, 69, 89]","[angularjs, asp.net-web-api, c#, html, javascr...","[c++, java, c#, gcc, javascript]"
27597,"[c, arrays, pointers, memory, language-lawyer]",2,"[12, 25, 97, 107, 129]","[89, 26, 76, 91, 87]","[arrays, c, language-lawyer, memory, pointers]","[javascript, c#, html, jquery, java]"
29101,"[c++, c++11, pointers, memory-management, memo...",2,"[27, 28, 108, 109, 129]","[89, 26, 76, 91, 87]","[c++, c++11, memory-leaks, memory-management, ...","[javascript, c#, html, jquery, java]"
30287,"[javascript, jquery, html, css, image]",2,"[40, 76, 79, 89, 91]","[89, 26, 76, 91, 87]","[css, html, image, javascript, jquery]","[javascript, c#, html, jquery, java]"


In [25]:
import sklearn.metrics as metrics

def metrics_score(model, df, y_true, y_pred):
    """
    Fonction de compilation des métriques spécifiques aux problèmes de classification multi-labels dans un DataFrame Pandas. Ce DataFrame aura 1 ligne par métrique et 1 colonne par modèle testé.

    Paramètres
    ----------------------------------------
    model : string
        Nom du modèle testé.
    df : DataFrame 
        DataFrame à étendre. 
        Si None : Créer un DataFrame.
    y_true : array
        Tableau des valeurs réelles à tester.
    y_pred : array
        Tableau des valeurs prédites à tester.
    ----------------------------------------
    
    Retour
    ----------------------------------------
    DataFrame
        Un DataFrame avec les scores des différentes métriques pour le modèle spécifié.
    """
    if df is not None:
        temp_df = df
    else:
        temp_df = pd.DataFrame(index=["Accuracy", "F1",
                                      "Jaccard", "Precision",
                                      "Recall"],
                               columns=[model])
        
    scores = []
    scores.append(metrics.accuracy_score(y_true, y_pred))
    scores.append(metrics.f1_score(y_true, y_pred, average='weighted'))
    scores.append(metrics.jaccard_score(y_true, y_pred, average='weighted'))
    scores.append(metrics.precision_score(y_true, y_pred, average='weighted'))
    scores.append(metrics.recall_score(y_true, y_pred, average='weighted'))
    temp_df[model] = scores
    
    return temp_df


In [26]:
# Création de la matrice y true et pred
lda_y_pred = np.zeros(y_binarized.shape)
n = 0
for row in y_results.y_pred.values:
    for i in range(len(row)):
        lda_y_pred[n,row[i]] = 1
    n+=1
    
lda_y_true = np.zeros(y_binarized.shape)
m = 0
for row in y_results.y_true.values:
    for i in range(len(row)):
        lda_y_true[m,row[i]] = 1
    m+=1

In [ ]:
df_metrics = metrics_score("LDA", df=None,
                                   y_true=lda_y_true,
                                   y_pred=lda_y_pred)
df_metrics